# Setup Environemnt

In [ ]:
import os

workspace_root = os.environ["SELF_DRIVE_CARLA_WORKSPACE"]
project_workspace = os.path.join(workspace_root, "01-semantic-segmentation")

In [ ]:
import torch
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
from torch.utils.tensorboard import SummaryWriter

# Import your SegmentationDataset class
from src.segmentation_dataset import SegmentationDataset

# Perform train test validation split

In [ ]:
import pandas as pd

full_dataset_path= os.path.join(project_workspace, "reduced_dataset.csv")
df = pd.read_csv(full_dataset_path, nrows=None)

### Train

In [ ]:
train_mask = [i % 10 < 7 for i in range(len(df))]
train_df = df[train_mask]
train_df.to_csv(os.path.join(project_workspace, "reduced_train_dataset.csv"), index=False)

### Validation

In [ ]:
val_mask = [7 <= i % 10 < 9 for i in range(len(df))]
val_df = df[val_mask]
val_df.to_csv(os.path.join(project_workspace, "reduced_val_dataset.csv"), index=False)

### Test

In [ ]:
test_mask = [i % 10 == 9 for i in range(len(df))]
test_df = df[test_mask]
test_df.to_csv(os.path.join(project_workspace, "reduced_test_dataset.csv"), index=False)

# Train

In [ ]:
n_classes = 15

# Define model (DeepLabV3+ with pre-trained encoder)
model = smp.DeepLabV3Plus(
    encoder_name="resnet34",  # Choose appropriate encoder (e.g., resnet50, efficientnet-b0)
    encoder_weights="imagenet",  # Load pre-trained weights from ImageNet
    classes=n_classes,  # Number of classes in your dataset
)

# Define loss function (e.g., CrossEntropyLoss for multi-class segmentation)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0) 

model = model.cuda()

In [ ]:
from torchvision import transforms

# Define a transform sequence for image data (adjust as needed)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize RGB channels
    # Add other transformations like random cropping, flipping, etc. (optional)
])

In [ ]:
train_dataset = SegmentationDataset(os.path.join(project_workspace, "reduced_train_dataset.csv"), project_workspace,mode="train", transform=transform)
val_dataset = SegmentationDataset(os.path.join(project_workspace, "reduced_val_dataset.csv"), project_workspace, transform=transform)

print(len(train_dataset))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

In [10]:
from torchmetrics.classification import MulticlassAccuracy
from torchmetrics.classification import MulticlassJaccardIndex

# Initialize the metrics
mean_accuracy = MulticlassAccuracy(ignore_index=0, num_classes=n_classes).cuda()
miou = MulticlassJaccardIndex(num_classes=n_classes).cuda()

writer = SummaryWriter(os.path.join(project_workspace, "runs/first_2k_try2"))

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epoch = 0

while epoch <= 50:
    model.train()
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()
        
        predictions = model(images)
        loss = loss_fn(predictions, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        mean_accuracy(predictions, labels)
        miou(predictions, labels)

    # Compute the metrics after all batches are processed
    mean_acc = mean_accuracy.compute()
    iou = miou.compute()

    # Log the metrics after each epoch
    writer.add_scalar('Loss', loss.item(), epoch)
    writer.add_scalar('Mean Accuracy', mean_acc, epoch)
    writer.add_scalar('Mean IoU', iou, epoch)

    # Save the model after each epoch
    torch.save(model.state_dict(), os.path.join(project_workspace, f"models/first_2k_try2/model_epoch_{epoch}.pth"))
    epoch += 1

writer.close()

In [ ]:
model.eval()    



test_image = val_dataset[7][0].unsqueeze(0).cuda()

with torch.no_grad():
    test_output = model(test_image)

predicted_mask = test_output.squeeze().argmax(dim=0)

In [ ]:
import matplotlib.pyplot as plt
from src.labels import trainId2label
import numpy as np

# Convert predicted mask to RGB image
predicted_mask_rgb = np.zeros((480, 640, 3))
for h in range(480):
  for w in range(640):
    class_label = predicted_mask[h, w]
    predicted_mask_rgb[h, w] = trainId2label[class_label.item()].color

predicted_mask_rgb = predicted_mask_rgb / 255.0

# Predicted mask (RGB)
plt.imshow(predicted_mask_rgb)
plt.axis("off")
plt.show()